In [275]:
%pylab inline
import dfi
import dfi.fasta_convert
import pandas as pd
import dfi.fastaseq 
from dfi.fastaseq import mapres
from __future__ import print_function
from __future__ import division

Populating the interactive namespace from numpy and matplotlib


In [276]:
def calc_hessian(x,y,z,Verbose=False):
    """
    Calculate the hessian given the coordinates 
    
    Input
    -----
    (x,y,z) numpy array
       Must all be the same length
    Output
    ------
    kirchhoff: NxN numpy matrix
       
    """
    cutoff = 10
    gamma = 1
    xyz = np.column_stack((x,y,z))
    numres = xyz.shape[0]
    kirchhoff = np.zeros((numres,numres))
    for i in range(numres):
        xyz_i = xyz[i]
        i_p1 = i + 1
        xyz_ij = xyz[i_p1:] - xyz_i
        xyz_ij2=np.multiply(xyz_ij,xyz_ij)
        cutoff2 = cutoff * cutoff
        for j, dist2 in enumerate(xyz_ij2.sum(1)):
            if dist2 > cutoff2:
                continue
            if Verbose:
                print(j, dist2)
            j += i_p1
            kirchhoff[i,j] = -gamma 
            kirchhoff[j,i] = -gamma 
            kirchhoff[i,i] += gamma 
            kirchhoff[j,j] += gamma
    return kirchhoff

In [277]:
# Invert this matrix
def invert_kirchhoff(kirchhoff):
    """
    Invert matrix 
    """
    from scipy import linalg as LA
    U, w, Vt = LA.svd(kirchhoff,full_matrices=False)
    S = LA.diagsvd(w,len(w),len(w))
    np.allclose(kirchhoff,np.dot(U,np.dot(S,Vt)))
    tol = 1e-6
    singular = w < tol
    assert np.sum(singular) == 1.
    invw = 1/w
    invw[singular] = 0
    inv_kirchhoff = np.dot(np.dot(U,np.diag(invw)),Vt)
    return inv_kirchhoff

###Use EC from EVfold to generate kirchhoff

In [117]:
def _build_kirchhoff(evod_file,n):
    """
    Creates a kirchoff matrix using EVfold contacts
    Input
    -----
    evfold input file: str
       file from evfold
    n: size of the square matrix
    Output
    ------
    kirchoff: NxN numpy matrix
       output matrix
    """

    chain = []
    chain_connection = np.zeros((n,n))
    
    #assign a -1 for residues in contact in the chain
    for i in range(3, n-3):
        chain_connection[i, i+1] = -1
        chain_connection[i, i+2] = -1
        chain_connection[i, i+3] = -1
        chain_connection[i+1, i] = -1
        chain_connection[i+2, i] = -1
        chain_connection[i+3, i] = -1
        chain_connection[i, i-1] = -1
        chain_connection[i, i-2] = -1
        chain_connection[i, i-3] = -1
        chain_connection[i-1, i] = -1
        chain_connection[i-2, i] = -1
        chain_connection[i-3, i] = -1
        
        chain.append([i, i+1, chain_connection[i, i+1]])
        chain.append([i, i+2, chain_connection[i, i+2]])
        chain.append([i, i+3, chain_connection[i, i+3]])
        chain.append([i+1, i, chain_connection[i+1, i]])
        chain.append([i+2, i, chain_connection[i+2, i]])
        chain.append([i+3, i, chain_connection[i+3, i]])
        chain.append([i, i-1, chain_connection[i, i-1]])
        chain.append([i, i-2, chain_connection[i, i-2]])
        chain.append([i, i-3, chain_connection[i, i-3]])
        chain.append([i-1, i, chain_connection[i-1, i]])
        chain.append([i-2, i, chain_connection[i-2, i]])
        chain.append([i-3, i, chain_connection[i-3, i]])
        
    #assign a -1 for EC pairs
    evol = []
    contact_pairs = open(evod_file, 'rU').readlines() 
    evol_const = np.zeros((n,n))
    for line in contact_pairs:
        a = line.split()
        i = int(a[0]) - 1 
        j = int(a[2]) - 1 
        if (chain_connection[i, j] != -1):
            evol_const[i, j] = -1.0*float(a[5])
            evol_const[j, i] = -1.0*float(a[5])
            evol.append([i, j, evol_const[i, j]])
            evol.append([j, i, evol_const[j, i]])
    
    #build kirchoff matrix
    kirchhoff = np.zeros((n,n))
    kirchhoff = chain_connection + evol_const
    print('generated kirchhoff using evolutionary constraints')
    print('kirchhoff shape: ', kirchhoff.shape)
    
    #calculate the diagonal
    diag = []
    for i in range(0, n):
        kirchhoff[i, i] = -np.sum(kirchhoff[i])
        diag.append([i, i, kirchhoff[i, i]])
    
    #put everything together for a file
    all = chain + evol + diag
    f = open('evfold_kirchhoff.txt', 'w')
    for x in all:
        f.write('%s \t %s \t %s \n' % (x[0], x[1], x[2]))
    f.close()
    
    return kirchhoff;

###Use EC from RaptorX server to generate kirchhoff

In [278]:
def _build_kirchhoff(evod_file,n):
    """
    Creates a kirchoff matrix using EVfold contacts
    Input
    -----
    evfold input file: str
       file from evfold
    n: size of the square matrix
    Output
    ------
    kirchoff: NxN numpy matrix
       output matrix
    """

    chain = []
    chain_connection = np.zeros((n,n))
    
    #assign a -1 for residues in contact in the chain
    for i in range(3, n-3):
        chain_connection[i, i+1] = -1
        chain_connection[i, i+2] = -1
        chain_connection[i, i+3] = -1
        chain_connection[i+1, i] = -1
        chain_connection[i+2, i] = -1
        chain_connection[i+3, i] = -1
        chain_connection[i, i-1] = -1
        chain_connection[i, i-2] = -1
        chain_connection[i, i-3] = -1
        chain_connection[i-1, i] = -1
        chain_connection[i-2, i] = -1
        chain_connection[i-3, i] = -1
        
        chain.append([i, i+1, chain_connection[i, i+1]])
        chain.append([i, i+2, chain_connection[i, i+2]])
        chain.append([i, i+3, chain_connection[i, i+3]])
        chain.append([i+1, i, chain_connection[i+1, i]])
        chain.append([i+2, i, chain_connection[i+2, i]])
        chain.append([i+3, i, chain_connection[i+3, i]])
        chain.append([i, i-1, chain_connection[i, i-1]])
        chain.append([i, i-2, chain_connection[i, i-2]])
        chain.append([i, i-3, chain_connection[i, i-3]])
        chain.append([i-1, i, chain_connection[i-1, i]])
        chain.append([i-2, i, chain_connection[i-2, i]])
        chain.append([i-3, i, chain_connection[i-3, i]])
        
    #assign a -1 for EC pairs
    evol = []
    contact_pairs = open(evod_file, 'rU').readlines() 
    evol_const = np.zeros((n,n))
    for line in contact_pairs:
        a = line.split()
        i = int(a[0]) - 1 
        j = int(a[1]) - 1 
        if (chain_connection[i, j] != -1):
            evol_const[i, j] = -1.0 #*float(a[4])
            evol_const[j, i] = -1.0 #*float(a[4])
            evol.append([i, j, evol_const[i, j]])
            evol.append([j, i, evol_const[j, i]])
    
    #build kirchoff matrix
    kirchhoff = np.zeros((n,n))
    kirchhoff = chain_connection + evol_const
    print('generated kirchhoff using evolutionary constraints')
    print('kirchhoff shape: ', kirchhoff.shape)
    
    #calculate the diagonal
    diag = []
    for i in range(0, n):
        kirchhoff[i, i] = -np.sum(kirchhoff[i])
        diag.append([i, i, kirchhoff[i, i]])
    
    #put everything together for a file
    all = chain + evol + diag
    f = open('evfold_kirchhoff.txt', 'w')
    for x in all:
        f.write('%s \t %s \t %s \n' % (x[0], x[1], x[2]))
    f.close()
    
    return kirchhoff;

In [279]:
#dfi.fetch_pdb('5pnt')
mdlpdbfile='5rsa.pdb'
exptpdbfile='5rsa.pdb'
evoDfile='5rsa_contactmap.txt'
uniprotID='P61823'

In [280]:
#PDB MODEL
ATOMS = dfi.pdbio.pdb_reader(mdlpdbfile,CAonly=True)
x,y,z = dfi.getcoords(ATOMS)
numres = len(x)
mdlseq=[mapres[atom.res_name] for atom in ATOMS]
kirchhoff = calc_hessian(x,y,z)
inv_kirchhoff = invert_kirchhoff(kirchhoff)
mdl_diag = np.array([inv_kirchhoff[i,i] for i in range(numres)])

Read 124 atoms from the 5rsa.pdb


In [281]:
#Experimental PDB 
expt_ATOM = dfi.pdbio.pdb_reader(exptpdbfile,CAonly=True)
expt_betafactors = np.array([atom.temp_factor for atom in expt_ATOM])
exptseq=[mapres[atom.res_name] for atom in expt_ATOM]

Read 124 atoms from the 5rsa.pdb


In [282]:
#EVFOLD Contacts 
str_seq=''.join( dfi.fastaseq.get_fastaseq(uniprotID).split('\n')[1:] )
fastaseq=[s for s in str_seq]
numseq = len(fastaseq)
evodkirchhoff=_build_kirchhoff(evoDfile,numseq)
inv_evodkirchhoff=invert_kirchhoff(evodkirchhoff)
evo_diag = np.array([inv_evodkirchhoff[i,i] for i in range(numseq)])

generated kirchhoff using evolutionary constraints
kirchhoff shape:  (150, 150)


# Need to align sequences properly

## Grab the 5pnt fasta sequence 

In [283]:
print(len(fastaseq))
print(len(mdlseq))
print(len(exptseq))

150
124
124


In [284]:
align={'fastaseq':''.join(fastaseq),
       'mdlseq':''.join(mdlseq),
       'exptseq':''.join(exptseq)}

In [285]:
from clustalo import clustalo

In [286]:
aligned=clustalo(align,seqtype=3)

In [287]:
dft = pd.DataFrame()

In [288]:
aligned

{'exptseq': '--------------------------KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTFVHESLADVQAVCSQKNVACKNGQTNCYQSYSTMSITDCRETGSSKYPNCAYKTTQANKHIIVACEGNPYVPVHFDASV',
 'fastaseq': 'MALKSLVLLSLLVLVLLLVRVQPSLGKETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTFVHESLADVQAVCSQKNVACKNGQTNCYQSYSTMSITDCRETGSSKYPNCAYKTTQANKHIIVACEGNPYVPVHFDASV',
 'mdlseq': '--------------------------KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTFVHESLADVQAVCSQKNVACKNGQTNCYQSYSTMSITDCRETGSSKYPNCAYKTTQANKHIIVACEGNPYVPVHFDASV'}

In [289]:
dft['EVfold_R'] = [s for s in aligned['fastaseq']]
dft['mdl_R'] = [s for s in aligned['mdlseq']]
dft['expt_R'] = [s for s in aligned['exptseq']]

In [290]:
dft.head()

,EVfold_R,mdl_R,expt_R
0,M,-,-
1,A,-,-
2,L,-,-
3,K,-,-
4,S,-,-


In [291]:
#align b-factors with the corresponding sequence 
seqtype='fastaR'
bfactype='fastaB'
evo_diag

def align_bfac(seqtype,bfactype,ls_bfac,dft):
    """
    
    """
    align_bfac = []
    i=0
    for r in dft[seqtype]:
        #print(r)
        if i >= len(ls_bfac):
            break
        if r == '-':
            align_bfac.append(np.nan)
        else:
            align_bfac.append(ls_bfac[i])
            i+=1
    while len(align_bfac) < len(dft[seqtype]):
        align_bfac.append(np.nan)

    dft[bfactype] = align_bfac

In [292]:
align_bfac('EVfold_R','EVfold_B',evo_diag,dft)
align_bfac('mdl_R','mdl_B',mdl_diag,dft)
align_bfac('expt_R','expt_B',expt_betafactors,dft)

In [293]:
dft

,EVfold_R,mdl_R,expt_R,EVfold_B,mdl_B,expt_B
0,M,-,-,1.504150,NaN,NaN
1,A,-,-,0.886915,NaN,NaN
2,L,-,-,0.661546,NaN,NaN
3,K,-,-,0.517483,NaN,NaN
4,S,-,-,0.427497,NaN,NaN
5,L,-,-,0.374752,NaN,NaN
6,V,-,-,0.329915,NaN,NaN
7,L,-,-,0.130792,NaN,NaN
8,L,-,-,0.258273,NaN,NaN
9,S,-,-,0.236668,NaN,NaN


In [294]:
dft.to_csv(exptpdbfile[0:4]+'_align.csv',index=False)

In [295]:
dft.dropna().corr()

,EVfold_B,mdl_B,expt_B
EVfold_B,1.000000,-0.058080,0.084201
mdl_B,-0.058080,1.000000,0.681232
expt_B,0.084201,0.681232,1.000000


In [296]:
#obtain correlation coefficients
correl = dft[[i for i in dft.columns if '_B' in i]].dropna().corr()
print(correl)
correl.to_csv(exptpdbfile[0:4]+'_corr.csv') 

          EVfold_B     mdl_B    expt_B
EVfold_B  1.000000 -0.058080  0.084201
mdl_B    -0.058080  1.000000  0.681232
expt_B    0.084201  0.681232  1.000000
